In [1]:


import colorsys
import os
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model
import time 

def non_max_suppression(boxes, max_bbox_overlap, scores=None):
   
    if len(boxes) == 0:
        return []

    boxes = boxes.astype(np.float)
    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    if scores is not None:
        idxs = np.argsort(scores)
    else:
        idxs = np.argsort(y2)

        
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        
        iou = w*h
        out = np.where(iou/area[i] > 0.8)

        
        idxs = np.delete(
            idxs, np.concatenate(
                ([last], np.where(overlap > max_bbox_overlap)[0],out[0])))

    return pick


class YOLO(object):
    _defaults = {
        "anchors_path": 'model_data/yolo_anchors.txt',
        "score" : 0.3,
        "iou" : 0.5,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        # model with dataset 600 and no Text button, no Back
#         model = 'C:/Users/ADMINS/Desktop/all_model/model/model_supervised_RL/version1_lr_1e-7_loss_14.h5' 
#         model = 'C:/Users/ADMINS/Desktop/version3_tiny.h5'
    
        model = 'C:/Users/ADMINS/Desktop/test.h5'
#         model = 'C:\\Users\\ADMINS\\Desktop\\all_model\\tiny_model\\trained_weights_v3.h5'
#         classes = 'C:\\Users\\ADMINS\\Desktop\\all_model\\tiny_model\\classes.txt'
        classes= 'C:/Users/ADMINS/Desktop/classes.txt'
        self.model_path = model
        self.classes_path = classes
           
           
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
                
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.
        

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image, show = False):

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        idx = non_max_suppression(out_boxes, 0.55, out_scores)
        out_boxes, out_scores, out_classes = out_boxes[idx], out_scores[idx], out_classes[idx]
        
        if show:
            font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                        size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
            thickness = (image.size[0] + image.size[1]) // 300
            
            for i, c in reversed(list(enumerate(out_classes))):
                predicted_class = self.class_names[c]
                box = out_boxes[i]
                score = out_scores[i]

                label = '{} {:.2f}'.format(predicted_class, score)
                draw = ImageDraw.Draw(image)
                label_size = draw.textsize(label, font)

                top, left, bottom, right = box
                top = max(0, np.floor(top + 0.5).astype('int32'))
                left = max(0, np.floor(left + 0.5).astype('int32'))
                bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
                right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
                # print(label, (left, top), (right, bottom))

                if top - label_size[1] >= 0:
                    text_origin = np.array([left-150, top - label_size[1]])
                else:
                    text_origin = np.array([left-150, top + 1])

                # My kingdom for a good redistributable image drawing library.
                for i in range(thickness):
                    draw.rectangle(
                        [left + i, top + i, right - i, bottom - i],
                        outline=self.colors[c])
                draw.rectangle(
                    [tuple(text_origin), tuple(text_origin + label_size)],
                    fill=self.colors[c])
                draw.text(text_origin, label, fill=(0, 0, 0), font=font)
                del draw

        return image, out_boxes, out_scores, out_classes

    def close_session(self):
        self.sess.close()





Using TensorFlow backend.
c:\users\admins\anaconda3\envs\ebox\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\admins\anaconda3\envs\ebox\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\admins\anaconda3\envs\ebox\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\admins\anaconda3\envs\ebox\lib\site-pa

In [2]:
yolo = YOLO()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
import os
import cv2
from PIL import Image
import numpy as np

path = 'C:/Users/ADMINS/Desktop/newest/'
for file in os.listdir(path):
#     print(file)
    img = Image.open(path+file)
    image,_,_,_ =  yolo.detect_image(img,show=True)
    image = np.array(image)
    image = image[:,:,::-1]
    cv2.imwrite('C:/Users/ADMINS/Desktop/small_data_result/'+file,image)


In [89]:
from difflib import SequenceMatcher
import pytesseract as pt
import numpy as np
import cv2

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def blackAndWhite(img):
    # take input is binary image, find the box contain text
    position = []
    temp = []
    found = 0
    for k,v in enumerate(img[:,0]):
        if k < 10 and v!=0:
            continue
        if found == 2:
            position.append(temp)
            temp = []
            found = 0
        try:
            gradient = np.abs(img[k,0]-img[k+1,0])
            if gradient > 0:
                temp.append(k)
                found+=1
        except:
            pass
    return position

def readTextBox(img, boxes, classes, labels):
    # class of text box is 6
    boxes = boxes[np.where(classes == 6)]
    
    # check if the box is empty or not
    try:
        if not boxes:
            return img
    except:
        pass
    
    boxes = np.round(boxes).astype(np.int64)
    
    # convert image to correct channels
#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # convert image to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # find box contain all text
    x_min = np.min(boxes[:,1])
    x_max = np.max(boxes[:,3])
    y_top = np.min(boxes[:,0])
    y_bot = np.max(boxes[:,2])
    
    # extract it from original image
    image = gray[y_top:y_bot,x_min:x_max]
    
    H,W = image.shape
#     return image
    # take threshold, set thresh very small because the word is always black
    ret,thresh = cv2.threshold(image,20,255,cv2.THRESH_BINARY_INV)
    # Openning horizontal to remove the small object
    # E.g: the tail of word 'g' 
    kernel = np.ones((3,1),np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    
    # Dilate horizontal to get the white box contain text
    kernel = np.ones((1,5000),np.uint8)
    dilation = cv2.dilate(opening,kernel,iterations = 1)
    
    # this part only use for show result
    color = [255,0,0]
    thickness = 3
    # plus 10 pixels to top and bottom to make sure it contain enough information 
    for p in blackAndWhite(dilation):
        i = image[(p[0]-10):(p[1]+10),:]
#         return i
        txt = pt.image_to_string(i)
        if txt == '':
            try:
                ret,thresh1 = cv2.threshold(i,50,255,cv2.THRESH_BINARY_INV)
                txt = pt.image_to_string(thresh1)
            except:
                pass
        max = -1
        for k,v in labels.items():
            if similar(v,txt) > max:
                max = similar(v,txt)
                idx = k
        
        cv2.rectangle(img,(x_min,p[0]+y_top-10),(x_max,p[1]+y_top+10), color, thickness)
        if max < 0.7:
            continue
        font = cv2.FONT_HERSHEY_SIMPLEX 
        cv2.putText(img, labels[idx], (x_min-500,p[0]+y_top), font, 1, [0,255,0], 2,cv2.LINE_AA)
    img = img[:,:,::-1]
    return img

    
            
            

In [93]:
import cv2
import time
import json 

with open('labels.json', 'r') as l:
    labels = json.loads(l.read())
# pic = Image.open('C:/Users/ADMINS/Desktop/image/(83).png')
pic = Image.open('C:/Users/ADMINS/Desktop/hari/image/1.png')
# pic = Image.open('C:/Users/ADMINS/Desktop/test.png')
image, boxes, scores, classes = yolo.detect_image(image = pic, show =False)
# print(pic.shape)
image = np.array(image)
# print(image.shape)
img = readTextBox(image, boxes, classes, labels)
cv2.imwrite('C:/Users/ADMINS/Desktop/fix.png', img)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 1000,600)

cv2.imshow('image',img)
cv2.waitKey()
cv2.destroyAllWindows()

c:\users\admins\anaconda3\envs\ebox\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in ubyte_scalars


In [94]:
import os
import time
import json
t = []
path = 'C:/Users/ADMINS/Desktop/image/'
path = 'C:/Users/ADMINS/Desktop/hari/image/'
# dictionary contain all label
with open('labels.json', 'r') as l:
    labels = json.loads(l.read())
    
for img in os.listdir(path):
    print(img)
    t1 = time.time()
    pic = Image.open(path+img)
    _, boxes, _, classes = yolo.detect_image(image = pic)
    cv2.imwrite('C:/Users/ADMINS/Desktop/fix/'+img, \
                readTextBox(np.array(pic), boxes, classes, labels))
    t.append(time.time()-t1)


1.png


c:\users\admins\anaconda3\envs\ebox\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in ubyte_scalars


10.png
11.png
2.png
3.png
4.png
5.png
6.png
7.png
8.png
9.png


In [89]:
a = np.array(t)
print(np.average(a))

1.1751042222976684


In [25]:
import pytesseract as pt
import cv2

# path = 'C:/Users/TobyCurtis/Desktop/(4).png'

# image = cv2.imread(path,0)
# image = image[y_top:y_bot,x_min:x_max]
# text = pt.image_to_string(image)
# print(text)

image = img
boxes = np.round(boxes).astype(np.int64)

ret,thresh = cv2.threshold(image,170,255,cv2.THRESH_BINARY_INV)

kernel = np.ones((3,1),np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# kernel = np.ones((1,3),np.uint8)
# opening = cv2.morphologyEx(opening, cv2.MORPH_OPEN, kernel)

kernel = np.ones((1,5000),np.uint8)
# closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
dilation = cv2.dilate(opening,kernel,iterations = 1)

cv2.imshow('w3', image)
# cv2.imshow('w2', opening)
cv2.imshow('w1',thresh)
# cv2.imshow('w',dilation)
cv2.waitKey()
cv2.destroyAllWindows()

In [40]:
t1 = 'Language Option English' 
t2 = 'Language Option'
similar(t1,t2)

0.7894736842105263

In [45]:
a = np.array(t)
print(np.average(a))

1.206016926765442


In [1]:
import cv2
import os
import time

goto = 'C:/Users/ADMINS/Desktop/Ebox/Source2/result/'
source = 'C:/Users/ADMINS/Desktop/Ebox/Source2/image/'
special_source = 'C:/Users/ADMINS/Desktop/Ebox/Source1/special_test/'
result = 'C:/Users/ADMINS/Desktop/Ebox/Source1/special_result/'

new_source = 'C:/Users/ADMINS/Desktop/Data_Test_image/'
new_result = 'C:/Users/ADMINS/Desktop/result1/'


# time_ex = []
iou_1 = []
iou_2 = []
iou_3 = []
iou_4 = []
iou_5 = []
for file in os.listdir(new_source):
    pic = Image.open(new_source+file)
    image, boxex, scores, classes = yolo.detect_image(image = pic)

    img = np.array(image)
    img = img[:,:,::-1]
    cv2.imwrite(new_result+file,img)
    

In [68]:
f = open('C:/Users/ADMINS/Desktop/Ebox/Source1/test.txt','r')
d = {}
for line in f:
    a = line
    temp_dict = {}
    b = a.split(' ')
    name = b[0].split('/')[-1]
    count = 1
    while(count<len(b)-1):
        value = [int(i) for i in b[count].split(',')]
        temp_dict[value[-1]]=value[:-1]
        count+=1
    d[name]=temp_dict


In [29]:
temp_dict = {}
b = a.split(' ')
name = b[0].split('/')
count = 1
while(count<len(b)-1):
    value = [int(i) for i in b[count].split(',')]
    temp_dict[value[-1]]=value[:-1]
    count+=1
print(temp_dict)

{0: [1198, 48, 1270, 119], 1: [1201, 152, 1267, 213], 2: [1204, 254, 1270, 317], 3: [1200, 354, 1270, 424], 4: [1202, 456, 1269, 520], 5: [1201, 561, 1268, 620]}


In [8]:
def iou(box_gt, box_pre):
    x_left = max(box_pre[1],box_gt[0])
    x_right = min(box_pre[3], box_gt[2])
    y_top = max(box_pre[0], box_gt[1])
    y_bot = min(box_pre[2], box_gt[3])
    if x_left > x_right:
        return 0
    intersect_area = (x_right-x_left)*(y_bot-y_top)
    
    gt_area = (box_gt[2]-box_gt[0])*(box_gt[3]-box_gt[1])
    pre_area = (box_pre[3]-box_pre[1])*(box_pre[2]-box_pre[0])
    
    result = intersect_area/(gt_area+pre_area-intersect_area)
    return result
box_pre = [  50, 1193,    113, 1273 ]
box_gt = [1198,48,1270,119]
iou(box_gt,box_pre)

0.8076923076923077

In [2]:


import colorsys
import os
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model

def non_max_suppression(boxes, max_bbox_overlap, scores=None):
   
    if len(boxes) == 0:
        return []

    boxes = boxes.astype(np.float)
    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    if scores is not None:
        idxs = np.argsort(scores)
    else:
        idxs = np.argsort(y2)

        
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        
        iou = w*h
        out = np.where(iou/area[i] > 0.8)

        
        idxs = np.delete(
            idxs, np.concatenate(
                ([last], np.where(overlap > max_bbox_overlap)[0],out[0])))

    return pick


class YOLO(object):
    _defaults = {
        "anchors_path": 'model_data/yolo_anchors.txt',
        "score" : 0.5,
        "iou" : 0.5,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        # model with dataset 600 and no Text button, no Back
        model1 = 'C:/Users/ADMINS/Desktop/model/first_model/trained_weights_final.h5'
        
        # model with dataset 100 with 8 labels
        # normal train, early stop
        model2 = 'C:/Users/ADMINS/Desktop/model/OMEN_Text_Button/trained_weights_final.h5'
        
        # no early, no decay
        model3 = 'C:/Users/ADMINS/Desktop/model/OMEN_model_non_early_non_decay/trained_weights_final.h5'
        
        class_6 = 'C:/Users/ADMINS/Desktop/Ebox/Source1/classes.txt'
        class_8 = 'C:/Users/ADMINS/Desktop/classes.txt'
        self.model_path = model3
        self.classes_path = class_8
           
           
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))
        
        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.
        

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image, show = False):

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        idx = non_max_suppression(out_boxes, 0.55, out_scores)
        out_boxes, out_scores, out_classes = out_boxes[idx], out_scores[idx], out_classes[idx]
        
        if show:
            font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                        size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
            thickness = (image.size[0] + image.size[1]) // 300
            
            for i, c in reversed(list(enumerate(out_classes))):
                predicted_class = self.class_names[c]
                box = out_boxes[i]
                score = out_scores[i]

                label = '{} {:.2f}'.format(predicted_class, score)
                draw = ImageDraw.Draw(image)
                label_size = draw.textsize(label, font)

                top, left, bottom, right = box
                top = max(0, np.floor(top + 0.5).astype('int32'))
                left = max(0, np.floor(left + 0.5).astype('int32'))
                bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
                right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
                # print(label, (left, top), (right, bottom))

                if top - label_size[1] >= 0:
                    text_origin = np.array([left-150, top - label_size[1]])
                else:
                    text_origin = np.array([left-150, top + 1])

                # My kingdom for a good redistributable image drawing library.
                for i in range(thickness):
                    draw.rectangle(
                        [left + i, top + i, right - i, bottom - i],
                        outline=self.colors[c])
                draw.rectangle(
                    [tuple(text_origin), tuple(text_origin + label_size)],
                    fill=self.colors[c])
                draw.text(text_origin, label, fill=(0, 0, 0), font=font)
                del draw

        return image, out_boxes, out_scores, out_classes

    def close_session(self):
        self.sess.close()





Using TensorFlow backend.
c:\users\admins\anaconda3\envs\ebox\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\admins\anaconda3\envs\ebox\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\admins\anaconda3\envs\ebox\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\admins\anaconda3\envs\ebox\lib\site-pa

In [3]:
yolo = YOLO()

C:/Users/ADMINS/Desktop/model/OMEN_model_non_early_non_decay/trained_weights_final.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
a = yolo.generate()

C:/Users/ADMINS/Desktop/model/OMEN_model_non_early_non_decay/trained_weights_final.h5 model, anchors, and classes loaded.


In [16]:
import numpy as np
np.array(yolo.yolo_model.output[0])

array(<tf.Tensor 'conv2d_209/BiasAdd:0' shape=(?, ?, ?, 39) dtype=float32>,
      dtype=object)

In [17]:
import h5py
f = h5py.File('C:/Users/ADMINS/Desktop/model/OMEN_model_non_early_non_decay/trained_weights_final.h5', 'r')
print(list(f.keys())

SyntaxError: unexpected EOF while parsing (<ipython-input-17-40f70cd6e643>, line 3)